In [87]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator
import random
import numpy as np
import math
from copy import deepcopy as dp

In [88]:
stateArr = np.array(['100/0','90/10','70/30','50/50','30/70','10/90','0,100','entangle','measure'])
valueArr = [100,90,70,50,30,10,0]
measureArr = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
pairArr = [[0,1],[0,2],[0,3],[0,4],[0,6],[0,8],[1,2],[1,4],[1,7],[2,4],[2,5],[2,6],[2,8],[3,4],[3,5],[3,6],[4,5],[4,6],[4,7],[4,8],[5,8],[6,7],[6,8],[7,8]]

In [89]:
def makeQuantumCircuit(m, measureIndexes, superMeasure):
    qc = QuantumCircuit(9,9)
    placeQubits(m, qc)
    EntangleQubits(m, qc)
    qc.measure(measureIndexes, measureIndexes)
    sum, o = simulate(qc)
    o = o[::-1]
    if superMeasure:
        measurable = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
        for m in measurable:
            b = (o[m[0]] == o[m[1]]) and (o[m[1]] == o[m[2]])
            if b:
                return b
        return False    
    else:
        return (sum == 3 or sum == 0)

    
    
def placeQubits(m, qc):
    for i in range(0,3):
        for j in range(0,3):
            if m[2][i][j] == 0:
                qc.unitary(makeOperator(m[0][i][j]), [3*i + j], label = str(m[0][i][j])) 
                  
            
def EntangleQubits(m, qc):  
    for i in range(0,3):
        for j in range(0,3):
            if m[2][i][j] >= 70 and m[2][i][j] < 80:
                qc.barrier()
                qc.cx(m[2][i][j] - 70, 3*i + j)
            if m[2][i][j] >= 80 and m[2][i][j] < 90:
                qc.barrier()
                qc.x(3*i + j)
                qc.cx(m[2][i][j] - 80, 3*i + j)
            
            
def makeOperator(probability):
    angle = math.acos(np.sqrt(probability/100))
    return Operator([
    [math.cos(angle),  math.sin(angle)],
    [-math.sin(angle), math.cos(angle)]
    ])

def simulate(qc):
    sim = Aer.get_backend('aer_simulator')
    transpiled = transpile(qc, sim)
    result = sim.run(transpiled, shots=1, memory = True).result()
    o = result.get_memory()
    sum = 0
    for indices in measureIndexes:
        sum = sum + int(o[8-indices])
    return (sum, o)

In [90]:
import numpy as np
from copy import deepcopy as dp


def Fork(Q):
    matrices = []
    weights = []
    N = [[(0,0),(0,0),(0,0)],[(0,0),(0,0),(0,0)],[(0,0),(0,0),(0,0)]]
    for i in range(0,3):
        for j in range(0,3):
            if(Q[0][i][j] == -1):
                N[i][j] = -1
            else:   
                N[i][j] = (Q[0][i][j]/100, Q[1][i][j]/100)
    func(1, N, matrices, weights)
    return matrices, weights

def func(weight, N, matrices, weights):
    flag = True
    for i in range(0,3):
        for j in range(0,3):
            if type(N[i][j]) == tuple:
                rightweight =(N[i][j])[0]
                leftweight = (N[i][j])[1]
                R = dp(N)
                L = dp(N)
                R[i][j] = 0
                L[i][j] = 1
                func(weight*rightweight, R, matrices, weights)
                func(weight*leftweight, L, matrices, weights)
                flag = False;
                break;
        if not flag:
            break 
    if flag:
        matrices.append(N)
        weights.append(weight)
        
    
    

In [94]:
def tripleScore (A):
    A = np.reshape(A,9)
    tripleScore = 0
    for measureAlong in measureArr:
        if (A[measureAlong[0]] == A[measureAlong[1]]) and (A[measureAlong[1]] == A[measureAlong[2]]):
            tripleScore += 1
    return tripleScore


def pairScore(A):
    A = np.reshape(A,9)
    pairScore = 0
    for pair in pairArr:
        if A[pair[0]] == A[pair[1]]:
            pairScore += 1
    return pairScore            
    
        
        
def decideClassicalMove(A,x,y):
        trialMatrix0 = A
        trialMatrix1 = A
        trialMatrix0[x][y] = 0
        ts0 = tripleScore(trialMatrix0)
        ps0 = pairScore(trialMatrix0)
        trialMatrix1[x][y] = 1
        ts1 = tripleScore(trialMatrix1)
        ps1 = pairScore(trialMatrix1)
        if ts1 > ts0:
            return 0
        elif ts0 > ts1:
            return 1
        else:
            if ps1 > ps0:
                return 1
            elif ps0 >= ps1:
                return 0
            
def decidePlayState(Q):
    matrices, weights = Fork(Q)
    empty_cells = []
    state = np.zeros(9)
    #print("initially state is ",state)
    print(len(matrices)==len(weights))
    for c1 in range (0,3):
        for c2 in range (0,3):
            if matrices[0][c1][c2] == -1:
                #print("inside main loop now: ",c1, c2)
                for r in range(0, len(matrices)):
                    weight = weights[r]
                    H = matrices[r]
                    state[3*c1 + c2] = state[3*c1 + c2] +  weight*decideClassicalMove(H, c1, c2)    
    
    print("state = ",state) 
    for i in range(0,9):
        if state[i] > 0:
            Q[0][4//3][4%3] = state[i]*100;
            Q[1][4//3][4%3] = 100 - 100*state[i];
    print(Q)
    return Q, 0
    
    
def decideMove(Q):
    M = np.reshape(Q[0,:,:],(9))
    threshold = 0.8
    threshCond = (100*threshold)**3
    for measureAlong in measureArr:
        product0 = 1
        product1 = 1
        for position in measureAlong:
            if M[position] != -1:
                product0 *= M[position]
                product1 *= 100 - M[position]
            else:
                product0 = 0
                product1 = 0
                
        findMove = True
        if product0 >= threshCond or product1 >= threshCond:
            print("yes please, measure along axis", measureAlong, "win probability is", max(product0,product1)/10000, "%")
            measureYes = measureAlong
            findMove = False
            return Q, measureYes
    
            
    if findMove:        
        print("Do not take the risk of measuring, play a move instead")
        A, l = decidePlayState(Q)
        return A, l


def userMove(N):
    P = N
    K = N[0,:,:]
    L = N[1,:,:]
    print("Current board:\n", K)
    M = np.reshape(K,9)
    U = np.reshape(L,9)
    print("Chose a move\nEnter 0 for placing a qubit\nEnter 1 for Making a Measurement")
    move = int(input())

    Repeat = True
    while Repeat == True:
        if move == 0:
            print("\nWhich cell would you like to place a qubit in\n0.Top left\n1.Center top\n2.Top right\n3.Left center\n4. Middle\n5. Right center\n6. Bottom left\n7. Bottom center\n8. Bottom right")
            cell = int(input())

            if M[cell] == -1:
                print("\nWhat value would you like to fill?\n0. 100/0\n1. 90/10\n2. 70/30\n3. 50/50\n4. 30/70\n5. 10/90\n6. 0/100\n7. Entangle positively\n8. Entangle negatively")
                
                valInput = int(input())
                if valInput <= 6:
                    value = valueArr[valInput]
                    M[cell] = value
                    U[cell] = 100 - value
                    Repeat = False
                    M = np.reshape(M,(3,3))
                    U = np.reshape(U,(3,3))
                    P[0,:,:] = M
                    P[1,:,:] = U
                    return P, move
                else:
                    print("Please select the particle with which you would like to entangle, an integer from 0 to 8 representing the cell")
                    ent1 = int(input())
                    if M[ent1] == -1:
                        print("Sorry, you cannot entangle with an empty cell")
                        repeat = True
                    entType = valInput*10 + ent1
                    repeat = False
                    #print(["entangle", cell, ent1, entType])
                    
                    P[2,cell//3,cell%3] = valInput*10 + ent1
                    
                    if valInput == 7:
                        P[0,cell//3,cell%3] = 100 - M[ent1]
                        P[1,cell//3,cell%3] = M[ent1]
                    elif valInput == 8:
                        P[0,cell//3,cell%3] = M[ent1]
                        P[1,cell//3,cell%3] = 100 - M[ent1]

                    return P,move


            else:
                print("Sorry, please select an empty cell")
                repeat = True


        else:
            print("\nPlease select line along which you would like to make the measurement\n0. Left column\n1. Middle column\n2. Right column\n3. Top row\n4. Middle row\n5. Bottom row\n6. Main diagonal\n7. Antidiagonal")
            measureAlong = measureArr[int(input())]
            print (["measure", measureAlong])
            return P, measureAlong

        




In [95]:
#main code

def main():
    A = np.array([[[-1,-1,-1],[-1,-1,-1],[-1,-1,-1]],
              [[-1,-1,-1],[-1,-1,-1],[-1,-1,-1]],
              [[0,0,0],[0,0,0],[0,0,0]]])
    
    A, measureAlong = userMove(A)
    print(A)
    gameOver = False
    while not gameOver:
        winner = ""
        turn = "computer"
        A, measureAlong = decideMove(A)
        if measureAlong != 0:
            gameOver, O = makeQuantumCircuit(A, measureAlong, False)
            if gameOver:
                winner = "computer"
        else: 
            turn = "human"
            A, measureAlong = userMove(A)
            if measureAlong != 0:
                gameOver, O = makeQuantumCircuit(A, measureAlong, False)
            if gameOver:
                winner = "human"
        if winner == "human":
            print("Congrats, you win!")
        elif winner == "computer":
            print("Haha, Quantum rules! Better luck next time!")


main()



Current board:
 [[-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]]
Chose a move
Enter 0 for placing a qubit
Enter 1 for Making a Measurement
0

Which cell would you like to place a qubit in
0.Top left
1.Center top
2.Top right
3.Left center
4. Middle
5. Right center
6. Bottom left
7. Bottom center
8. Bottom right
2

What value would you like to fill?
0. 100/0
1. 90/10
2. 70/30
3. 50/50
4. 30/70
5. 10/90
6. 0/100
7. Entangle positively
8. Entangle negatively
0
[[[ -1  -1 100]
  [ -1  -1  -1]
  [ -1  -1  -1]]

 [[ -1  -1   0]
  [ -1  -1  -1]
  [ -1  -1  -1]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
Do not take the risk of measuring, play a move instead
True
state =  [0. 0. 0. 1. 1. 0. 0. 0. 0.]
[[[ -1  -1 100]
  [ -1 100  -1]
  [ -1  -1  -1]]

 [[ -1  -1   0]
  [ -1   0  -1]
  [ -1  -1  -1]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
Current board:
 [[ -1  -1 100]
 [ -1 100  -1]
 [ -1  -1  -1]]
Chose a move
Enter 0 for placing a qubit
Enter 1 for Making a Measurement
0

Which cell wou

C:\Users\hp\AppData\Local\Temp/ipykernel_30432/1324613608.py:40: RuntimeWarning: invalid value encountered in sqrt
  angle = math.acos(np.sqrt(probability/100))


ExtensionError: 'Input matrix is not unitary.'